# **Case Datarisk - Score de Crédito**

## **Importa as bibliotecas e Carrega os Dados**

In [2]:
from utils.RankCountVectorizer import RankCountVectorizer
from utils.load_df import load_df
from utils.convert_to_datetime import convert_to_datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker

import requests
from pathlib import Path

from sklearn.metrics import roc_curve

In [3]:
# Carregar os DataFrames
targets = load_df("./data/targets.parquet")

df_cadastral = load_df("./data/base_cadastral.parquet")
df_emprestimos = load_df("./data/historico_emprestimos.parquet")
df_submissao = load_df("./data/base_submissao.parquet")
df_parcelas = load_df("./data/historico_parcelas.parquet")
dicionario = load_df("./data/dicionario_dados.csv")

## **Converter Datas**

In [4]:
# listar as colunas que precisam de conversão em cada DataFrame
colunas_data_emprestimos = [
    'data_decisao',
    'data_liberacao',
    'data_primeiro_vencimento',
    'data_ultimo_vencimento_original',
    'data_ultimo_vencimento',
    'data_encerramento'
]

colunas_data_parcelas = [
    'data_prevista_pagamento',
    'data_real_pagamento'
]

colunas_data_cadastral = [
    'data_nascimento'
]

df_parcelas = convert_to_datetime(df_parcelas, colunas_data_parcelas)
df_emprestimos = convert_to_datetime(df_emprestimos, colunas_data_emprestimos)
df_cadastral = convert_to_datetime(df_cadastral, colunas_data_cadastral)


--- Verificando tipos de dados ANTES da conversão ---
data_prevista_pagamento    object
data_real_pagamento        object
dtype: object

--- Verificando tipos de dados DEPOIS da conversão ---
data_prevista_pagamento    datetime64[ns]
data_real_pagamento        datetime64[ns]
dtype: object

--- Verificando tipos de dados ANTES da conversão ---
data_decisao                       object
data_liberacao                     object
data_primeiro_vencimento           object
data_ultimo_vencimento_original    object
data_ultimo_vencimento             object
data_encerramento                  object
dtype: object

--- Verificando tipos de dados DEPOIS da conversão ---
data_decisao                       datetime64[ns]
data_liberacao                     datetime64[ns]
data_primeiro_vencimento           datetime64[ns]
data_ultimo_vencimento_original    datetime64[ns]
data_ultimo_vencimento             datetime64[ns]
data_encerramento                  datetime64[ns]
dtype: object

--- Verificando t

## **Préprocessamento**

In [5]:
# Calcular a idade dos clientes
# Assumindo a data atual como referência para o cálculo da idade
data_atual_idade = pd.to_datetime('today')
df_cadastral['idade_cliente'] = ((data_atual_idade - df_cadastral['data_nascimento']).dt.days / 365.25).astype(int)

# Remover a coluna data_nascimento
df_cadastral = df_cadastral.drop(columns=['data_nascimento'])

In [6]:
df_cadastral['reda_por_familiar'] = df_cadastral['renda_anual'] / df_cadastral['qtd_membros_familia']

In [7]:
targets['inadimplente'].value_counts()

inadimplente
0    103007
1      4412
Name: count, dtype: int64

In [8]:
include_columns = [
                  'id_contrato',
                  'id_cliente',
                  'dia_semana_solicitacao',
                  'hora_solicitacao',
                  'tipo_contrato',
                  'valor_credito',
                  'valor_bem',
                  'valor_parcela',
                  'data_decisao'
                  ]

df = df_emprestimos.copy()
df = df[include_columns]

In [9]:
df = df.merge(df_cadastral, on='id_cliente', how='left')
df = df.merge(targets, on='id_contrato', how='left')

In [10]:
df.sort_values('data_decisao', inplace=True)

In [11]:
df.dropna(subset=['inadimplente'], inplace=True)

In [12]:
df['inadimplente'].value_counts()

inadimplente
0.0    103007
1.0      4412
Name: count, dtype: int64

Extração de Features
- Features Cadastrais:
  - idade_cliente: Idade do cliente (diferença em anos entre a data atual e a data de nascimento do cliente)
  - renda_por_familiar: Quociente entre a renda anual do cliente e a quantidade de membros da sua família
  - compormetimento_de_renda: Quociente entre o valor do crédito e a renda anual do cliente
- Features históricas:
  - num_emp_aceitos_6m: Número de empréstimos solicitados pelo cliente que foram aceitos nos últimos 6 meses
  - atraso_medio: Média do número de dias atrasados por cliente até o momento

In [13]:
df['comprometimento_renda'] = df['valor_credito'] / df['renda_anual']

In [14]:
df_prev_apps = df_emprestimos[['id_cliente', 'data_decisao', 'status_contrato']].copy()
df_prev_apps['data_decisao'] = pd.to_datetime(df_prev_apps['data_decisao'])
df_prev_apps.rename(columns={'data_decisao': 'data_decisao_prev'}, inplace=True)

# Filter out 'Refused' contracts
df_prev_apps = df_prev_apps[df_prev_apps['status_contrato'] != 'Refused'].copy()
df_prev_apps.drop(columns=['status_contrato'], inplace=True) # Drop the status_contrato column after filtering

# Merge the main dataframe with previous applications
df_merged = pd.merge(df, df_prev_apps, on='id_cliente', how='left')

# Calculate the time difference between the current decision date and previous decision dates
df_merged['time_diff'] = (df_merged['data_decisao'] - df_merged['data_decisao_prev']).dt.days

# Filter previous applications within the last 6 months (180 days) and before the current decision date
df_filtered = df_merged[(df_merged['time_diff'] > 0) & (df_merged['time_diff'] <= 180)].copy()

# Count the number of filtered previous applications for each current contract
prev_apps_count = df_filtered.groupby('id_contrato').size().reset_index(name='num_emp_aceitos_6m')

# Merge the count back to the main dataframe
df = pd.merge(df, prev_apps_count, on='id_contrato', how='left')

# Fill NaN values (for contracts with no previous applications in the last 6 months) with 0
df['num_emp_aceitos_6m'].fillna(0, inplace=True)

# Convert the new column to integer type
df['num_emp_aceitos_6m'] = df['num_emp_aceitos_6m'].astype(int)

/var/folders/py/_hms0lwn3h70z03wxhxl24jm0000gp/T/ipykernel_10001/798354359.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['num_emp_aceitos_6m'].fillna(0, inplace=True)


In [15]:
# Merge df_parcelas with df to get data_decisao for each contract
df_prev_atraso = pd.merge(df_parcelas, df[['id_contrato', 'data_decisao']], on='id_contrato', how='left')

# Filter parcels to include only those before the contract's data_decisao
df_prev_atraso_filtered = df_prev_atraso[df_prev_atraso['data_prevista_pagamento'] < df_prev_atraso['data_decisao']].copy()

# Calculate the delay, clipping at 0 for early payments
df_prev_atraso_filtered['atraso'] = (df_prev_atraso_filtered['data_real_pagamento'] - df_prev_atraso_filtered['data_prevista_pagamento']).dt.days.clip(lower=0)

# Calculate the mean delay for each client based on filtered parcels
df_prev_atraso_agg = df_prev_atraso_filtered.groupby('id_cliente')['atraso'].mean().reset_index()

# Rename the column and merge back to the main dataframe
df_prev_atraso_agg.rename(columns={'atraso': 'atraso_medio'}, inplace=True)
df = pd.merge(df, df_prev_atraso_agg, on='id_cliente', how='left')

# Fill NaN values (for clients with no previous parcels before data_decisao) with 0
df['atraso_medio'].fillna(0, inplace=True)

/var/folders/py/_hms0lwn3h70z03wxhxl24jm0000gp/T/ipykernel_10001/445545720.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['atraso_medio'].fillna(0, inplace=True)


In [16]:
drop_columns = ['sexo', 'data_decisao', 'id_contrato', 'id_cliente']

df = df.drop(columns = drop_columns)

### **Separação entre treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

# Separar variáveis explicativas e target
y = df['inadimplente']
X = df.drop('inadimplente', axis=1)

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:
cat_cols = X_train.select_dtypes(include=['object']).columns

In [19]:
rankVectorizer = RankCountVectorizer()

X_train = rankVectorizer.fit_transform(X_train, cols=cat_cols)
X_test = rankVectorizer.transform(X_test, cols=cat_cols)

/Users/jvlm/repositories/case-ds/src/utils/RankCountVectorizer.py:107: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(-999, inplace=True)


In [21]:
df_train = X_train.copy()
df_train['inadimplente'] = y_train

df_test = X_test.copy()
df_test['inadimplente'] = y_test

df_train.to_parquet("./data/train.parquet", index=False)
df_test.to_parquet("./data/test.parquet", index=False)